# ARIMA Forecasting

## Importing Libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random

from datetime import date
from dateutil.relativedelta import relativedelta

from keras import optimizers
from keras import Sequential
from keras.layers import LSTM, Dropout
from tensorflow.keras import layers
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.models import load_model

Using TensorFlow backend.


## Reading Data from Snowflake

In [2]:
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

USER = "PC_USER"
PASSWORD = "Edge@123"
ACCOUNT = "edgematics.eu-west-1"
WAREHOUSE = "TAL_SF_WH"
DATABASE = "TAL_SF"
SCHEMA = "PUBLIC"

engine = create_engine(
    URL(account=ACCOUNT,
        user=USER,
        password=PASSWORD,
        warehouse=WAREHOUSE,
        database=DATABASE,
        schema=SCHEMA))

table_name = "customer_acquisition"

sql = f"SELECT * FROM {DATABASE}.{SCHEMA}.{table_name}"
with engine.connect() as conn:
    df = pd.read_sql_query(sql, con=conn)
print(df.head())
df.info()
df.to_csv('customer_acquisition.csv')

  activation_date  circle_id    emirate_name            subs_key      msisdn  \
0      2014-11-03          4        Fujairah  050260011120141103  0502600111   
1      2014-11-03          1       Abu Dhabi  056984621520141103  0569846215   
2      2014-11-03          6         Sharjah  056821731120141103  0568217311   
3      2014-11-03          5  Ras Al Khaimah  050322789520141103  0503227895   
4      2014-11-03          3           Dubai  050728068020141103  0507280680   

  acquisition_region      acquisition_pos  pre_post_ind customer_type  \
0           Fujairah        Galleria Mall             2    Enterprise   
1          Abu Dhabi       Abu Dhabi Mall             1    Enterprise   
2            Sharjah        Sahara Centre             1    Enterprise   
3     Ras Al Khaimah           Manar Mall             1        Retail   
4              Dubai  Dubai Festival City             1        Retail   

  acquisition_sales_channel_key  ... disconnection_date  \
0                    

In [3]:
# df['day']=pd.DatetimeIndex(df['activation_date']).date
df['year']=pd.DatetimeIndex(df['activation_date']).year
df['month']=pd.DatetimeIndex(df['activation_date']).month
df['year']=pd.DatetimeIndex(df['activation_date']).year
df['ad'] = pd.to_datetime(df['activation_date']).dt.to_period('M')
df.ad=df.ad.values.astype('datetime64[ns]')

migrtype = pd.get_dummies(df['migration_type'], prefix = 'migrtype')
migrtype['month']=df['month']
migrtype['year']=df['year']
migrtype=pd.concat([migrtype,df['ad']],axis=1)

attr=migrtype.columns[1]
group = migrtype.groupby(['ad'],as_index=False)[attr].sum()
group.index=group.ad

y = group[attr]
X=group['ad']

n=len(y)
length=6
length1=int(n/length)

samples=[]
xx=[]

xx=np.arange(length*length1).reshape(-1,length,1)
data=np.array(y).reshape(-1,length,1)

X_train, X_test, y_train, y_test = train_test_split(xx, data, shuffle=False, test_size=0.1)

mpredict=6
ar=np.arange(len(X),len(X)+mpredict).reshape(1,mpredict,1)
X_test=np.vstack((X_test,ar))
X_test.shape

startdate=X[0]

six_months = startdate + relativedelta(months=len(X_train.flatten())+len(X_test.flatten())-1)
enddate = six_months.strftime("%m-%Y")
startdate = startdate.strftime("%m-%Y")

X_labels=pd.date_range(startdate,enddate, freq='MS').strftime("%Y-%b").tolist()
X_labels = [str(x) for x in X_labels]
X_tr_labels=X_labels[:len(X_train.flatten())]
X_test_labels=X_labels[len(X_train.flatten()):]


## Forecasting

In [ ]:
# random.seed(42)
np.random.seed(25)
model = Sequential()
model.add(LSTM(128, activation='tanh', input_shape=(length, 1),return_sequences=True))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=25, activation='relu'))
model.add(Dense(units=12, activation='relu'))
model.add(Dense(units=6, activation='relu'))
model.add(Dense(units=3, activation='relu'))
model.add(Dense(units=1, activation='linear'))
adam = optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='mse')
model.fit(X_train, y_train, epochs=2100)
# model.save_weights(attr+'model.h5')
# model.load_weights(attr+'model.h5')
# model.save(attr+"model")

model_yaml = model.to_yaml()
with open(attr+"model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights(attr+"model.h5")
print("Saved model to disk")

# later...
from keras.models import model_from_yaml
# load YAML and create model
yaml_file = open(attr+'model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
model = model_from_yaml(loaded_model_yaml)
# load weights into new model
model.load_weights(attr+"model.h5")
print("Loaded model from disk")

Epoch 1/2100
10/10 [==============================] - 1s 75ms/step - loss: 204.8730
Epoch 2/2100
10/10 [==============================] - 0s 997us/step - loss: 204.3003
Epoch 3/2100
10/10 [==============================] - 0s 898us/step - loss: 203.8035
Epoch 4/2100
10/10 [==============================] - 0s 898us/step - loss: 203.2133
Epoch 5/2100
10/10 [==============================] - 0s 1ms/step - loss: 202.3828
Epoch 6/2100
10/10 [==============================] - 0s 798us/step - loss: 201.5143
Epoch 7/2100
10/10 [==============================] - 0s 1ms/step - loss: 200.5830
Epoch 8/2100
10/10 [==============================] - 0s 1ms/step - loss: 199.5751
Epoch 9/2100
10/10 [==============================] - 0s 1ms/step - loss: 198.4863
Epoch 10/2100
10/10 [==============================] - 0s 1ms/step - loss: 197.2849
Epoch 11/2100
10/10 [==============================] - 0s 997us/step - loss: 195.9568
Epoch 12/2100
10/10 [==============================] - 0s 1ms/step - loss:

Epoch 99/2100
10/10 [==============================] - 0s 901us/step - loss: 32.2258
Epoch 100/2100
10/10 [==============================] - 0s 901us/step - loss: 31.9518
Epoch 101/2100
10/10 [==============================] - 0s 1ms/step - loss: 31.6792
Epoch 102/2100
10/10 [==============================] - 0s 903us/step - loss: 31.4061
Epoch 103/2100
10/10 [==============================] - 0s 1ms/step - loss: 31.1314
Epoch 104/2100
10/10 [==============================] - 0s 1ms/step - loss: 30.8594
Epoch 105/2100
10/10 [==============================] - 0s 1ms/step - loss: 30.5955
Epoch 106/2100
10/10 [==============================] - 0s 998us/step - loss: 30.3360
Epoch 107/2100
10/10 [==============================] - 0s 1ms/step - loss: 30.0815
Epoch 108/2100
10/10 [==============================] - 0s 949us/step - loss: 29.8321
Epoch 109/2100
10/10 [==============================] - 0s 856us/step - loss: 29.5843
Epoch 110/2100
10/10 [==============================] - 0s 997us/

Epoch 196/2100
10/10 [==============================] - 0s 997us/step - loss: 16.5630
Epoch 197/2100
10/10 [==============================] - 0s 1ms/step - loss: 16.4266
Epoch 198/2100
10/10 [==============================] - 0s 850us/step - loss: 16.2903
Epoch 199/2100
10/10 [==============================] - 0s 1ms/step - loss: 16.1569
Epoch 200/2100
10/10 [==============================] - 0s 901us/step - loss: 16.0284
Epoch 201/2100
10/10 [==============================] - 0s 1ms/step - loss: 15.8988
Epoch 202/2100
10/10 [==============================] - 0s 808us/step - loss: 15.7695
Epoch 203/2100
10/10 [==============================] - 0s 798us/step - loss: 15.6408
Epoch 204/2100
10/10 [==============================] - 0s 1ms/step - loss: 15.5160
Epoch 205/2100
10/10 [==============================] - 0s 1ms/step - loss: 15.3927
Epoch 206/2100
10/10 [==============================] - 0s 1ms/step - loss: 15.2724
Epoch 207/2100
10/10 [==============================] - 0s 898us/s

Epoch 293/2100
10/10 [==============================] - 0s 898us/step - loss: 10.7112
Epoch 294/2100
10/10 [==============================] - 0s 1ms/step - loss: 10.6888
Epoch 295/2100
10/10 [==============================] - 0s 1ms/step - loss: 10.6590
Epoch 296/2100
10/10 [==============================] - 0s 1ms/step - loss: 10.6165
Epoch 297/2100
10/10 [==============================] - 0s 798us/step - loss: 10.5772
Epoch 298/2100
10/10 [==============================] - 0s 997us/step - loss: 10.5392
Epoch 299/2100
10/10 [==============================] - 0s 997us/step - loss: 10.5082
Epoch 300/2100
10/10 [==============================] - 0s 997us/step - loss: 10.4775
Epoch 301/2100
10/10 [==============================] - 0s 1ms/step - loss: 10.4409
Epoch 302/2100
10/10 [==============================] - 0s 1ms/step - loss: 10.4047
Epoch 303/2100
10/10 [==============================] - 0s 898us/step - loss: 10.3610
Epoch 304/2100
10/10 [==============================] - 0s 1ms/s

Epoch 390/2100
10/10 [==============================] - 0s 909us/step - loss: 5.2591
Epoch 391/2100
10/10 [==============================] - 0s 997us/step - loss: 5.2290
Epoch 392/2100
10/10 [==============================] - 0s 798us/step - loss: 5.2055
Epoch 393/2100
10/10 [==============================] - 0s 997us/step - loss: 5.1545
Epoch 394/2100
10/10 [==============================] - 0s 901us/step - loss: 5.0977
Epoch 395/2100
10/10 [==============================] - 0s 1ms/step - loss: 5.0374
Epoch 396/2100
10/10 [==============================] - 0s 1ms/step - loss: 5.0191
Epoch 397/2100
10/10 [==============================] - 0s 997us/step - loss: 4.9864
Epoch 398/2100
10/10 [==============================] - 0s 1ms/step - loss: 4.9287
Epoch 399/2100
10/10 [==============================] - 0s 997us/step - loss: 4.8851
Epoch 400/2100
10/10 [==============================] - 0s 1ms/step - loss: 4.8372
Epoch 401/2100
10/10 [==============================] - 0s 798us/step - l

...

Epoch 684/2100
10/10 [==============================] - 0s 901us/step - loss: 2.5455
Epoch 685/2100
10/10 [==============================] - 0s 896us/step - loss: 2.4682
Epoch 686/2100
10/10 [==============================] - 0s 898us/step - loss: 2.4266
Epoch 687/2100
10/10 [==============================] - 0s 898us/step - loss: 2.4073
Epoch 688/2100
10/10 [==============================] - 0s 998us/step - loss: 2.4313
Epoch 689/2100
10/10 [==============================] - 0s 898us/step - loss: 2.4665
Epoch 690/2100
10/10 [==============================] - 0s 1ms/step - loss: 2.4007
Epoch 691/2100
10/10 [==============================] - 0s 1ms/step - loss: 2.3605
Epoch 692/2100
10/10 [==============================] - 0s 1ms/step - loss: 2.3853
Epoch 693/2100
10/10 [==============================] - 0s 801us/step - loss: 2.3935
Epoch 694/2100
10/10 [==============================] - 0s 801us/step - loss: 2.3782
Epoch 695/2100
10/10 [==============================] - 0s 900us/step -

Epoch 782/2100
10/10 [==============================] - 0s 898us/step - loss: 2.1958
Epoch 783/2100
10/10 [==============================] - 0s 997us/step - loss: 2.2129
Epoch 784/2100
10/10 [==============================] - 0s 1ms/step - loss: 2.2400
Epoch 785/2100
10/10 [==============================] - 0s 798us/step - loss: 2.2729
Epoch 786/2100
10/10 [==============================] - 0s 898us/step - loss: 2.2920
Epoch 787/2100
10/10 [==============================] - 0s 850us/step - loss: 2.2939
Epoch 788/2100
10/10 [==============================] - 0s 751us/step - loss: 2.2709
Epoch 789/2100
10/10 [==============================] - 0s 1ms/step - loss: 2.2419
Epoch 790/2100
10/10 [==============================] - 0s 997us/step - loss: 2.2077
Epoch 791/2100
10/10 [==============================] - 0s 1ms/step - loss: 2.1844
Epoch 792/2100
10/10 [==============================] - 0s 1ms/step - loss: 2.1827
Epoch 793/2100
10/10 [==============================] - 0s 849us/step - l

Epoch 880/2100
10/10 [==============================] - 0s 901us/step - loss: 2.0660
Epoch 881/2100
10/10 [==============================] - 0s 901us/step - loss: 2.0743
Epoch 882/2100
10/10 [==============================] - 0s 901us/step - loss: 2.0768
Epoch 883/2100
10/10 [==============================] - 0s 901us/step - loss: 2.0725
Epoch 884/2100
10/10 [==============================] - 0s 1ms/step - loss: 2.0706
Epoch 885/2100
10/10 [==============================] - 0s 849us/step - loss: 2.0624
Epoch 886/2100
10/10 [==============================] - 0s 1ms/step - loss: 2.0526
Epoch 887/2100
10/10 [==============================] - 0s 1ms/step - loss: 2.0443
Epoch 888/2100
10/10 [==============================] - 0s 1ms/step - loss: 2.0388
Epoch 889/2100
10/10 [==============================] - 0s 997us/step - loss: 2.0371
Epoch 890/2100
10/10 [==============================] - 0s 904us/step - loss: 2.0325
Epoch 891/2100
10/10 [==============================] - 0s 997us/step - l

Epoch 978/2100
10/10 [==============================] - 0s 1ms/step - loss: 1.8923
Epoch 979/2100
10/10 [==============================] - 0s 798us/step - loss: 1.8934
Epoch 980/2100
10/10 [==============================] - 0s 1ms/step - loss: 1.8916
Epoch 981/2100
10/10 [==============================] - 0s 901us/step - loss: 1.8909
Epoch 982/2100
10/10 [==============================] - 0s 1ms/step - loss: 1.8889
Epoch 983/2100
10/10 [==============================] - 0s 898us/step - loss: 1.8860
Epoch 984/2100
10/10 [==============================] - 0s 849us/step - loss: 1.8814
Epoch 985/2100
10/10 [==============================] - 0s 803us/step - loss: 1.8793
Epoch 986/2100
10/10 [==============================] - 0s 1ms/step - loss: 1.8761
Epoch 987/2100
10/10 [==============================] - 0s 1ms/step - loss: 1.8752
Epoch 988/2100
10/10 [==============================] - 0s 902us/step - loss: 1.8717
Epoch 989/2100
10/10 [==============================] - 0s 901us/step - los

Epoch 1075/2100
10/10 [==============================] - 0s 1ms/step - loss: 1.7584
Epoch 1076/2100
10/10 [==============================] - 0s 1ms/step - loss: 1.7567
Epoch 1077/2100
10/10 [==============================] - 0s 1ms/step - loss: 1.7548
Epoch 1078/2100
10/10 [==============================] - 0s 1ms/step - loss: 1.7531
Epoch 1079/2100
10/10 [==============================] - 0s 901us/step - loss: 1.7519
Epoch 1080/2100
10/10 [==============================] - 0s 901us/step - loss: 1.7502
Epoch 1081/2100
10/10 [==============================] - 0s 901us/step - loss: 1.7491


In [ ]:
y_pred = model.predict(X_test)
y_pred=y_pred.flatten()
y_test=y_test.flatten()
y_train=y_train.flatten()
X_test=X_test.flatten()

In [ ]:
fig = plt.figure(figsize = (12,6))
ax = fig.add_subplot(111)
ax.set_xticks(np.arange(len(X_test_labels)))
ax.set_xticklabels(X_test_labels, rotation = 45)

plt.plot(y_pred,'-.')
plt.plot(y_test,'-.')

In [ ]:
y_tr_pred = model.predict(X_train)
y_tr_pred = y_tr_pred.flatten()

fig = plt.figure(figsize = (20,6))
ax = fig.add_subplot(111)
ax.set_xticks(np.arange(len(X_tr_labels)))
ax.set_xticklabels(X_tr_labels, rotation = 45)

plt.plot(y_train,'-.')
plt.plot(y_tr_pred,'-.')